In [1]:
import os
import numpy as np
from itertools import product
from scipy.signal import fftconvolve, convolve

# Convolutions

In [401]:
x = np.arange(10*10*10*10).reshape(10,10, 10, 10)

In [394]:
%%timeit
np.flipud(np.fliplr(x))

The slowest run took 9.67 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 1.46 µs per loop


In [402]:
%%timeit
flip_ndarray(x)

The slowest run took 8.19 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 1.83 µs per loop


In [2]:
def flip_ndarray(x):
    loc = tuple(slice(None, None,-1) for i in xrange(x.ndim))
    return x[loc]

In [3]:
def my_convd(dat, kernel, stride=1, pad=0):
    """ Perform a dot-product convolution of a n-dim kernel with n-dim data"""
    assert stride > 0
    assert dat.ndim == kernel.ndim
    stride = int(round(stride))
    
    outshape = (np.array(dat.shape)-np.array(kernel.shape)+2.*pad)/stride+1.
    for num in outshape:
        assert (num).is_integer(), num
    outshape = np.round(outshape).astype(int)
    
    if pad:
        dat = np.pad(dat, pad, mode='constant')

    out = np.zeros(outshape, dtype=dat.dtype)
    indices = range(dat.ndim)
    for i,x in enumerate(product(*[stride*np.arange(i) for i in outshape])):
        slices = tuple(slice(start, start+kernel.shape[i]) for i,start in enumerate(x))
        loc = np.unravel_index(i, outshape)
        out[loc] = np.einsum(dat[slices], indices, kernel, indices)

    return out

In [28]:
def old_multi_layer_conv(data, kernel, stride, pad, channels=True):
    if data.ndim == kernel.ndim:
        return my_convd(data, kernel, stride, pad)
    dat = data[0]
    assert stride > 0
    assert dat.ndim == kernel.ndim
    stride = int(round(stride))
    
    npad = np.array([0]+[pad for i in xrange(dat.ndim-1)])
    outshape = (np.array(dat.shape)-np.array(kernel.shape)+2.*npad)/float(stride)+1.
    for num in outshape:
        assert (num).is_integer(), num
    outshape = np.round(outshape).astype(int)
    
    total_out_shape = tuple([data.shape[0]]+[i for i in outshape])
    total_out = np.zeros(total_out_shape, dtype=data.dtype)
    
    indices = range(dat.ndim)
    all_pos = list(product(*[stride*np.arange(i) for i in outshape]))
    all_slices = [tuple(slice(start, start+kernel.shape[i]) for i,start in enumerate(x)) for x in all_pos]
    
    if pad and channels:
        pad = [(0,0)] + [(pad, pad) for i in xrange(dat.ndim-1)]
    for n, dat in enumerate(data):
        if pad:
            dat = np.pad(dat, pad, mode='constant').astype(dat.dtype)
        out = np.zeros(outshape, dtype=data.dtype)
        for i,slices in enumerate(all_slices):
            loc = np.unravel_index(i, outshape)
            out[loc] = np.einsum(dat[slices], indices, kernel, indices)
        total_out[n] = out
    return total_out

In [ ]:
def = get_outshape(dat, kernel, stride, channels=True):
    assert stride > 0
    assert dat.ndim == kernel.ndim
    stride = int(round(stride))
    outshape = (np.array(dat.shape)-np.array(kernel.shape))/stride+1.
    for num in outshape:
        assert (num).is_integer(), num
    outshape = np.round(outshape).astype(int)
    
    return outshape
    

In [5]:
def fast_convolve(dat, kernel, stride, pad, flip_kernel=True, outshape=None, channels=True):
    if np.max(dat.shape) >= 500:
        conv = fftconvolve
    else:
        conv = convolve
    if outshape is None:
        assert stride > 0
        assert dat.ndim == kernel.ndim
        stride = int(round(stride))
        outshape = (np.array(dat.shape)-np.array(kernel.shape)+2.*pad)/stride+1.
        for num in outshape:
            assert (num).is_integer(), num
        outshape = np.round(outshape).astype(int)
        
        if channels:
            outshape[0] = 1
        print outshape
    
    all_pos = zip(*product(*(stride*np.arange(i) for i in outshape)))
    out = np.zeros(outshape, dtype=dat.dtype)
    
    if pad:
        if channels:
            pad = [(0,0)] + [(pad, pad) for i in xrange(dat.ndim-1)]
        dat = np.pad(dat, pad, mode='constant').astype(dat.dtype)
    
    if flip_kernel:
        kernel = flip_ndarray(kernel)
    full_conv = conv(dat, kernel, mode='valid')
    
    if stride == 1:
        return full_conv
    
    out.flat = full_conv[all_pos]
    return out

In [6]:
def back_prop_dw(data, dy, w, stride, pad):
    if flip_kernel:
        kernel = flip_ndarray(kernel)
    for n, dat in enumerate(data):
        if n == 0:  # initialize output and stride information

            if outshape is None:
                assert dat.ndim == kernel.ndim
                stride = int(round(stride))

                npad = np.array([0]+[pad for i in xrange(dat.ndim-1)])
                outshape = (np.array(dat.shape)-np.array(kernel.shape)+2.*npad)/float(stride)+1.
                
                for num in outshape:
                    assert (num).is_integer(), num
                outshape = np.round(outshape).astype(int)
            
            if pad:
                pad = [(0, 0)] + [(pad, pad) for i in xrange(dat.ndim-1)]


        indices = range(dat.ndim)
        all_pos = list(product(*[stride*np.arange(i) for i in outshape]))
        all_slices = [tuple(slice(start, start+kernel.shape[i]) for i,start in enumerate(x)) for x in all_pos]
        
        if pad:
            dat = np.pad(dat, pad, mode='constant').astype(dat.dtype)
            
        out = np.zeros(outshape, dtype=data.dtype)
        dw = np.zeros_like(w, dtype=w.dtype)
        for i,slices in enumerate(all_slices):
            loc = np.unravel_index(i, outshape)
            dw += dy[loc]*out[slices]
        
    return dw

In [7]:
def back_prop_dx(data, dy, kernel, stride, pad, outshape=None):
    if flip_kernel:
        kernel = flip_ndarray(kernel)
    for n, dat in enumerate(data):
        if n == 0:  # initialize output and stride information

            if outshape is None:
                assert dat.ndim == kernel.ndim
                stride = int(round(stride))

                npad = np.array([0]+[pad for i in xrange(dat.ndim-1)])
                outshape = (np.array(dat.shape)-np.array(kernel.shape)+2.*npad)/float(stride)+1.
                
                for num in outshape:
                    assert (num).is_integer(), num
                outshape = np.round(outshape).astype(int)
            
            if pad:
                pad = [(0, 0)] + [(pad, pad) for i in xrange(dat.ndim-1)]

        total_out_shape = data.shape
        total_out = np.zeros(total_out_shape, dtype=dat.dtype)
        indices = range(dat.ndim)
        all_pos = list(product(*[stride*np.arange(i) for i in outshape]))
        all_slices = [tuple(slice(start, start+kernel.shape[i]) for i,start in enumerate(x)) for x in all_pos]
         
        out = np.zeros(outshape, dtype=data.dtype)
        for i,slices in enumerate(all_slices):
            loc = np.unravel_index(i, outshape)
            out[slices] += dy[loc]*kernel
        if pad:
            out = out[:, pad:-pad, pad:-pad]
        total_out[n] = out
    return total_out

In [434]:
x = np.arange(2).reshape(2,2)

In [540]:
range(4)[1:-1]

[1, 2]

In [439]:
np.pad(x, [(1,1),(0,0)], 'constant')

array([[0, 0],
       [0, 1],
       [2, 3],
       [0, 0]])

In [8]:
def multi_layer_conv(data, kernel, stride, pad, flip_kernel=True, outshape=None, channels=True):
    if flip_kernel:
        kernel = flip_ndarray(kernel)
    for n, dat in enumerate(data):
        if n == 0:  # initialize output and stride information

            if np.max(dat.shape) >= 500:
                conv = fftconvolve  # use fft method for large input arrays
            else:
                conv = convolve

            if outshape is None:
                assert dat.ndim == kernel.ndim
                stride = int(round(stride))

                npad = np.array([0]+[pad for i in xrange(dat.ndim-1)])
                outshape = (np.array(dat.shape)-np.array(kernel.shape)+2.*npad)/float(stride)+1.
                
                for num in outshape:
                    assert (num).is_integer(), num
                outshape = np.round(outshape).astype(int)
            
            if pad:
                if channels:
                    pad = [(0, 0)] + [(pad, pad) for i in xrange(dat.ndim-1)]

            total_out_shape = tuple([data.shape[0]]+[i for i in outshape])
            total_out = np.zeros(total_out_shape, dtype=dat.dtype)


            all_pos = zip(*product(*(stride*np.arange(i) for i in outshape)))


        if pad:
            dat = np.pad(dat, pad, mode='constant').astype(dat.dtype)

        full_conv = conv(dat, kernel, mode='valid')
        if stride == 1:
            total_out[n] = full_conv
        else:
            out = np.zeros(outshape, dtype=full_conv.dtype)
            out.flat = full_conv[all_pos]
            total_out[n] = out
    return total_out

In [536]:
all_pos = zip(*product(*(2*np.arange(i) for i in (2,3,4))))

In [537]:
all_pos

[(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2),
 (0, 0, 0, 0, 2, 2, 2, 2, 4, 4, 4, 4, 0, 0, 0, 0, 2, 2, 2, 2, 4, 4, 4, 4),
 (0, 2, 4, 6, 0, 2, 4, 6, 0, 2, 4, 6, 0, 2, 4, 6, 0, 2, 4, 6, 0, 2, 4, 6)]

In [508]:
dat = np.arange(1,17, dtype=float).reshape(4,4)
kernel = np.ones((2,2), dtype=float)
dat = np.arange(250*250*3, dtype=float).reshape(250,250,3)
kernel = np.random.rand(*(2,2,3))
stride = 2
pad = 0

outshape = (np.array(dat.shape)-np.array(kernel.shape)+2.*pad)/stride+1.
for num in outshape:
    assert (num).is_integer(), num
outshape = np.round(outshape).astype(int)
all_pos = zip(*product(*(stride*np.arange(i) for i in outshape)))

In [541]:
dat = np.arange(5*5*3,dtype=float).reshape(3,5,5)
kernel = np.ones((3,3,3), dtype=float)
stride = 2
pad = 1
# big = n.concatenate([dat, 2*dat]).reshape(-1,4,4)

In [45]:
x = np.random.rand(3,7,7)

(3, 3, 3)

In [510]:
w = np.random.rand(2,3,4,4)
x = np.random.rand(3,3,4,4)
conv_param = {'pad':1,'stride': 2}

In [29]:
dat = np.arange(1,17).reshape(4,4)
kernel = np.ones((4,2))
big = np.concatenate([dat, 2*dat]).reshape(-1,4,4)

In [36]:
multi_layer_conv(big, kernel, 1, 1)[0].shape

(1, 5)

In [32]:
old_multi_layer_conv(big,kernel,1,1)

array([[[ 28,  60,  68,  76,  40]],

       [[ 56, 120, 136, 152,  80]]])

In [535]:
tmp = 1. + (np.array(x.shape[2:]) + 2*conv_param['pad'] - np.array(w.shape[2:]))/conv_param['stride']
out = np.zeros((x.shape[0], w.shape[0], int(round(tmp[0])), int(round(tmp[1]))), dtype = float)
print x.shape
print w.shape
print conv_param
print out.shape
outshape = None
for n, kernel in enumerate(w):
    print n
    new = multi_layer_conv(x, kernel, conv_param['stride'], conv_param['pad'], outshape=outshape, channels=True)
    print new.shape
    out[:, n:n+1, :, :] = multi_layer_conv(x, kernel, conv_param['stride'], conv_param['pad'], outshape=outshape, channels=True)

#     outshape = out[:, n, :, :].shape
    
#     out[:, n, :, :] += b[n]

(3, 3, 4, 4)
(2, 3, 4, 4)
{'stride': 2, 'pad': 1}
(3, 2, 2, 2)
0
(3, 1, 2, 2)
1
(3, 1, 2, 2)


In [523]:
new

array([[[[ 6.48260056,  5.57298688],
         [ 7.4158236 ,  6.92183953]]],


       [[[ 7.07337669,  6.99860426],
         [ 7.10489032,  5.67299177]]],


       [[[ 4.4244933 ,  4.50384234],
         [ 4.99216288,  4.38133357]]]])

In [533]:
out[:, n:n+1, :, :] = new 

In [534]:
out

array([[[[ 6.48260056,  5.57298688],
         [ 7.4158236 ,  6.92183953]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]],


       [[[ 7.07337669,  6.99860426],
         [ 7.10489032,  5.67299177]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]],


       [[[ 4.4244933 ,  4.50384234],
         [ 4.99216288,  4.38133357]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]]])

In [379]:
big.ndim

3

In [380]:
multi_layer_conv(big, kernel, stride, pad)

array([[[   1.,    3.,    5.,    7.,    4.],
        [   6.,   14.,   18.,   22.,   12.],
        [  14.,   30.,   34.,   38.,   20.],
        [  22.,   46.,   50.,   54.,   28.],
        [  13.,   27.,   29.,   31.,   16.]],

       [[   2.,    6.,   10.,   14.,    8.],
        [  12.,   28.,   36.,   44.,   24.],
        [  28.,   60.,   68.,   76.,   40.],
        [  44.,   92.,  100.,  108.,   56.],
        [  26.,   54.,   58.,   62.,   32.]]])

In [381]:
fast_multi_layer_conv(big, kernel, stride, pad)

array([[[   1.,    3.,    5.,    7.,    4.],
        [   6.,   14.,   18.,   22.,   12.],
        [  14.,   30.,   34.,   38.,   20.],
        [  22.,   46.,   50.,   54.,   28.],
        [  13.,   27.,   29.,   31.,   16.]],

       [[   2.,    6.,   10.,   14.,    8.],
        [  12.,   28.,   36.,   44.,   24.],
        [  28.,   60.,   68.,   76.,   40.],
        [  44.,   92.,  100.,  108.,   56.],
        [  26.,   54.,   58.,   62.,   32.]]])

In [71]:
num_test = 1000
for i in range(num_test):
    ndim = np.random.randint(1, 6)
#     ndim=1
    shape = tuple(np.random.randint(1, 4) for i in range(ndim))
    k_shape = tuple(np.random.randint(1,1+dim) for dim in shape)
    dat = np.random.rand(*shape)
    kernel = np.random.rand(*k_shape)
    
    if not np.allclose(my_convd(dat, kernel, stride=1, pad=0), fftconvolve(dat, flip_ndarray(kernel), 'valid')):
        print dat.shape
        print kernel.shape
        print np.mean(np.absolute(my_convd(dat, kernel, stride=1, pad=0)-fftconvolve(dat, kernel, 'valid')))